# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings

warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize words
    tokens = word_tokenize(text)
    # Stem word tokens and remove stop words
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")
    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)
np.random.seed(17)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def get_eval_metrics(actual, predicted, col_names):
    metrics = []
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i],average='micro')
        recall = recall_score(actual[:, i], predicted[:, i],average='micro')
        f1 = f1_score(actual[:, i], predicted[:, i],average='micro')
        metrics.append([accuracy, precision, recall, f1])
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
    return metrics_df

In [7]:
Y_train_pred = pipeline.predict(X_train)

In [8]:
col_names = list(Y.columns.values)

In [9]:
print(get_eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                        Accuracy  Precision    Recall        F1
related                 0.990133   0.990133  0.990133  0.990133
request                 0.985556   0.985556  0.985556  0.985556
offer                   0.998576   0.998576  0.998576  0.998576
aid_related             0.985505   0.985505  0.985505  0.985505
medical_help            0.988455   0.988455  0.988455  0.988455
medical_products        0.991964   0.991964  0.991964  0.991964
search_and_rescue       0.994202   0.994202  0.994202  0.994202
security                0.994863   0.994863  0.994863  0.994863
military                0.994965   0.994965  0.994965  0.994965
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.994761   0.994761  0.994761  0.994761
food                    0.995219   0.995219  0.995219  0.995219
shelter                 0.992371   0.992371  0.992371  0.992371
clothing                0.998067   0.998067  0.998067  0.998067
money                   0.995423   0.995

In [10]:
Y_test_pred = pipeline.predict(X_test)
eval_metrics0 = get_eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.804852   0.804852  0.804852  0.804852
request                 0.884956   0.884956  0.884956  0.884956
offer                   0.996033   0.996033  0.996033  0.996033
aid_related             0.740769   0.740769  0.740769  0.740769
medical_help            0.917760   0.917760  0.917760  0.917760
medical_products        0.950259   0.950259  0.950259  0.950259
search_and_rescue       0.975130   0.975130  0.975130  0.975130
security                0.982148   0.982148  0.982148  0.982148
military                0.971010   0.971010  0.971010  0.971010
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.949802   0.949802  0.949802  0.949802
food                    0.941562   0.941562  0.941562  0.941562
shelter                 0.933781   0.933781  0.933781  0.933781
clothing                0.988099   0.988099  0.988099  0.988099
money                   0.978792   0.978

In [11]:
# %1's in each column
Y.sum()/len(Y)

related                   0.773650
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
def performance_metric(y_true, y_pred):
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i],average='micro')
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score
scorer = make_scorer(performance_metric)

In [13]:
parameters = {'vect__min_df': [1,5], 
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[10,20]} 

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10)
np.random.seed(81)
tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9572779981690571, total=  46.9s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9597192554165396, total=  46.8s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9586512053707659, total=  46.7s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  8.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.9592615196826366, total=  46.2s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  9.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.959948123283491, total=  42.4s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9598718339945072, total=  42.2s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9617790662191028, total=  42.3s
[CV] clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=10, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.9594140982606042, total= 1.4min
[CV] clf__estimator__min_samples_split=10

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed: 63.8min finished


In [14]:
tuned_model.cv_results_

{'mean_fit_time': array([ 37.8937494 ,  35.01973263,  37.3113958 ,  33.75725969,
         72.17757106,  64.89516195,  72.89476164,  62.07438962,
         35.71436334,  33.59232283,  35.06916642,  32.52657755,
         67.34892114,  62.39501651,  65.97053504,  59.50273808]),
 'std_fit_time': array([ 0.10574229,  0.14756196,  0.07295737,  0.07791195,  0.22079381,
         0.21720615,  1.83756113,  0.09895021,  0.19397604,  0.09847833,
         0.19352334,  0.13366347,  0.30896581,  0.03333596,  0.23709961,
         0.21741493]),
 'mean_score_time': array([  8.91983136,   8.57536467,   8.93122292,   8.51299644,
         11.29501049,  10.43147182,  11.320304  ,  10.26699106,
          8.92978414,   8.54431685,   8.87297002,   8.53371493,
         11.28384678,  10.42721057,  11.25614293,  10.29069869]),
 'std_score_time': array([ 0.06552128,  0.06155412,  0.10324944,  0.03601376,  0.05747619,
         0.02683135,  0.1324048 ,  0.07568489,  0.04733536,  0.09052739,
         0.07907213,  0.07

In [15]:
np.max(tuned_model.cv_results_['mean_test_score'])

0.96233852100498418

In [16]:
tuned_model.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 25,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
tuned_pred_test = tuned_model.predict(X_test)
eval_metrics1 = get_eval_metrics(np.array(Y_test), tuned_pred_test, col_names)
print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.821483   0.821483  0.821483  0.821483
request                 0.893042   0.893042  0.893042  0.893042
offer                   0.996033   0.996033  0.996033  0.996033
aid_related             0.774794   0.774794  0.774794  0.774794
medical_help            0.919133   0.919133  0.919133  0.919133
medical_products        0.951480   0.951480  0.951480  0.951480
search_and_rescue       0.975130   0.975130  0.975130  0.975130
security                0.982759   0.982759  0.982759  0.982759
military                0.971773   0.971773  0.971773  0.971773
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.962161   0.962161  0.962161  0.962161
food                    0.946292   0.946292  0.946292  0.946292
shelter                 0.941715   0.941715  0.941715  0.941715
clothing                0.988251   0.988251  0.988251  0.988251
money                   0.978944   0.978

In [18]:
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.944572,0.944572,0.944572,0.944572
std,0.058509,0.058509,0.058509,0.058509
min,0.740769,0.740769,0.740769,0.740769
25%,0.935307,0.935307,0.935307,0.935307
50%,0.957736,0.957736,0.957736,0.957736
75%,0.984551,0.984551,0.984551,0.984551
max,1.000000,1.000000,1.000000,1.000000


In [19]:
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.948937,0.948937,0.948937,0.948937
std,0.052291,0.052291,0.052291,0.052291
min,0.774794,0.774794,0.774794,0.774794
25%,0.944805,0.944805,0.944805,0.944805
50%,0.962618,0.962618,0.962618,0.962618
75%,0.985085,0.985085,0.985085,0.985085
max,1.000000,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
from sklearn.ensemble import AdaBoostClassifier
# Create a updated RandomForestClassifier with random_state pipeline with Classifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters2 = {'vect__min_df': [1,5], #1,5
               'tfidf__use_idf':[True, False],
               'clf__estimator__n_estimators':[15,25]
              } 

In [21]:
cv2 = GridSearchCV(pipeline2, param_grid = parameters2, scoring = scorer, verbose = 10)
np.random.seed(81)
tuned_model2 = cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1, score=0.9652883735123589, total=  40.8s
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   56.5s remaining:    0.0s


[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1, score=0.9651357949343912, total=  40.5s
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.9min remaining:    0.0s


[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=1, score=0.9645254806225205, total=  40.8s
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  7.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=False, vect__min_df=1, score=0.9636100091547146, total=  38.7s
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  8.2min remaining:    0.0s


[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=False, vect__min_df=5, score=0.9653646628013427, total=  36.0s
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=False, vect__min_df=5, score=0.9646780592004882, total=  36.2s
[CV] clf__estimator__n_estimators=15, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__n_estimators=15, tfidf__use_idf=False, vect__min_df=5, score=0.9635337198657309, total=  36.3s
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.965669819957278, total=  53.4s
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.9646780592004882, total=  53.0s
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__n_estimators=25, tf

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 24.0min finished


In [22]:
tuned_model2.cv_results_

{'mean_fit_time': array([ 32.81378555,  30.13204209,  30.83274802,  28.27428174,
         44.89051032,  40.49973981,  41.99026847,  37.3639733 ]),
 'std_fit_time': array([ 0.1827561 ,  0.05599761,  0.14780355,  0.112093  ,  0.1346796 ,
         0.04620197,  0.27525646,  0.08395356]),
 'mean_score_time': array([ 7.85963241,  7.93796802,  7.88095403,  7.86688995,  8.28390519,
         8.24300853,  8.41327477,  8.27569326]),
 'std_score_time': array([ 0.04427277,  0.11229089,  0.01873229,  0.09837914,  0.02127967,
         0.0484349 ,  0.13455387,  0.0837286 ]),
 'param_clf__estimator__n_estimators': masked_array(data = [15 15 15 15 25 25 25 25],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True True False False True True False False],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_vect__min_df': masked_array(data = [1 5 1 5 1 5 1 5],


In [23]:
tuned_pred_test2 = tuned_model2.predict(X_test)
eval_metrics2 = get_eval_metrics(np.array(Y_test), tuned_pred_test2, col_names)
print(eval_metrics2)

                        Accuracy  Precision    Recall        F1
related                 0.754196   0.754196  0.754196  0.754196
request                 0.877479   0.877479  0.877479  0.877479
offer                   0.995575   0.995575  0.995575  0.995575
aid_related             0.741379   0.741379  0.741379  0.741379
medical_help            0.925084   0.925084  0.925084  0.925084
medical_products        0.953616   0.953616  0.953616  0.953616
search_and_rescue       0.975282   0.975282  0.975282  0.975282
security                0.982148   0.982148  0.982148  0.982148
military                0.971620   0.971620  0.971620  0.971620
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.960940   0.960940  0.960940  0.960940
food                    0.942630   0.942630  0.942630  0.942630
shelter                 0.943546   0.943546  0.943546  0.943546
clothing                0.990845   0.990845  0.990845  0.990845
money                   0.979249   0.979

In [24]:
from sklearn.ensemble import AdaBoostClassifier
# Create a updated RandomForestClassifier with random_state pipeline with Classifier
pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters3 = {'vect__min_df': [1], #1,5
               'tfidf__use_idf':[False],
               'clf__estimator__n_estimators':[25]
              } 

In [25]:
cv3 = GridSearchCV(pipeline3, param_grid = parameters3, scoring = scorer, verbose = 10)
np.random.seed(81)
tuned_model3 = cv3.fit(X_train, Y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 
[CV]  clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.9660512664021972, total=  51.1s
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.9653646628013427, total=  50.2s
[CV] clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.2min remaining:    0.0s


[CV]  clf__estimator__n_estimators=25, tfidf__use_idf=False, vect__min_df=1, score=0.9644491913335368, total=  50.5s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.4min finished


In [26]:
tuned_model3.cv_results_

{'mean_fit_time': array([ 42.23919892]),
 'std_fit_time': array([ 0.33011276]),
 'mean_score_time': array([ 8.33901278]),
 'std_score_time': array([ 0.07305561]),
 'param_clf__estimator__n_estimators': masked_array(data = [25],
              mask = [False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [False],
              mask = [False],
        fill_value = ?),
 'param_vect__min_df': masked_array(data = [1],
              mask = [False],
        fill_value = ?),
 'params': [{'clf__estimator__n_estimators': 25,
   'tfidf__use_idf': False,
   'vect__min_df': 1}],
 'split0_test_score': array([ 0.96605127]),
 'split1_test_score': array([ 0.96536466]),
 'split2_test_score': array([ 0.96444919]),
 'mean_test_score': array([ 0.96528837]),
 'std_test_score': array([ 0.00065627]),
 'rank_test_score': array([1], dtype=int32),
 'split0_train_score': array([ 0.96647086]),
 'split1_train_score': array([ 0.96643271]),
 'split2_train_score': array([ 0.96677601]),
 'mean_tr

In [27]:
tuned_pred_test3 = tuned_model3.predict(X_test)
eval_metrics3 = get_eval_metrics(np.array(Y_test),tuned_pred_test3, col_names)
print(eval_metrics3)

                        Accuracy  Precision    Recall        F1
related                 0.754196   0.754196  0.754196  0.754196
request                 0.877479   0.877479  0.877479  0.877479
offer                   0.995575   0.995575  0.995575  0.995575
aid_related             0.741379   0.741379  0.741379  0.741379
medical_help            0.925084   0.925084  0.925084  0.925084
medical_products        0.953616   0.953616  0.953616  0.953616
search_and_rescue       0.975282   0.975282  0.975282  0.975282
security                0.982148   0.982148  0.982148  0.982148
military                0.971620   0.971620  0.971620  0.971620
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.960940   0.960940  0.960940  0.960940
food                    0.942630   0.942630  0.942630  0.942630
shelter                 0.943546   0.943546  0.943546  0.943546
clothing                0.990845   0.990845  0.990845  0.990845
money                   0.979249   0.979

### 9. Export your model as a pickle file

In [28]:
pickle.dump(tuned_model3, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.